# Re-allocating GBM wealth distribution model

In this notebook, we simulate 100,000 wealth trajectories following the model detailed in [_Wealth Inequality and the Ergodic Hypothesis: Evidence from the United States_](http://ssrn.com/abstract=2794830) (Adamou, Berman, Peters 2019).  
Briefly put, individuals start at a base wealth level of $1$. At each time step, their wealth $x$ grows following:

$$dx = x([\mu - \tau]dt + \sigma dW) + \tau \langle x \rangle_N dt$$

where $\tau$ is a proportion of their wealth that is added to a central pot, and then the pot is split evenly across the population ($\langle x \rangle_N$ beign the average wealth of the population).  
You are encouraged to try out different values for $\tau$ and explore the wealth distributions it generates.

Further details on the RGBM model are found in this blog post https://ergodicityeconomics.com/2017/08/14/wealth-redistribution-and-interest-rates/ 

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
plt.style.use('seaborn-notebook')
plt.rcParams['figure.figsize'] = (14, 8)

In [3]:
STEP_SIZE = 10
FRAMES = 100


def create_animation(tau=0.01):
    trajectories = generate_wealth_trajectories(tau)
    steps = trajectories.shape[1]
    
    fig = plt.figure()
    ax = plt.axes()
    ax.set_xlabel('Individual', fontsize=20)
    ax.set_ylabel('RGBM wealth', fontsize=20)
    
    bars = ax.bar(np.arange(steps), np.zeros(steps))

    # Initial wealth level
#     ax.axhline(1, alpha=0.3, linestyle='--', color='r')

    def animate(frame_number):
        i_day = frame_number * STEP_SIZE
        ax.set_ylim((0, 1.2 * max(trajectories.iloc[i_day])))
        bars.set_label('Year ' + str(i_day // 10 + 1))
        ax.legend(loc='upper right', fontsize=20)

        for individual, wealth in trajectories.iloc[i_day].iteritems():
            bars[individual].set_height(wealth)
        return bars

    anim = animation.FuncAnimation(fig,
                                   animate,
                                   frames=FRAMES,
                                   interval=500,
                                   blit=True);
    return anim

In [4]:
def generate_wealth_trajectories(tau=0.01):
    np.random.seed(42)
    N = 100
    T = 100
    dt = 0.1
    mu = 0.08
    sigma = 0.08
    time_steps = (int(T / dt))
    sdt = np.sqrt(dt)

    # Initially everyone has wealth 1
    x = np.zeros((time_steps, N))
    x[0][:] = 1

    # Generate noise array
    xi = np.random.normal(loc=0, scale=1, size=(time_steps, N))

    #Generate wealth trajectories
    for t in range(1, time_steps):
        x[t] = x[t - 1] * (1 + mu * dt + sigma * xi[t] * sdt) - tau * (
            x[t - 1] - np.mean(x[t - 1])) * dt

    return pd.DataFrame(x)

In [15]:
slider = widgets.FloatSlider(value=0.01,
                             min=-1.0,
                             max=1.0,
                             step=0.01,
                             description='Choose τ',
                             continuous_update=False,
                             readout=True,
                             readout_format='.2f')

display(slider)

FloatSlider(value=0.01, continuous_update=False, description='Choose τ', max=1.0, min=-1.0, step=0.01)

In [18]:
%%capture
anim = create_animation(tau=slider.value)

In [19]:
# In order to save the animation as a video file, you'll need FFMPEG (https://www.ffmpeg.org/)
# anim.save('rgbm.mp4', writer=animation.FFMpegWriter(fps=2))

# To save as a GIF ImageMagick (https://www.imagemagick.org) is required
# anim.save('rgbm.gif', dpi=80, writer='imagemagick')

HTML(anim.to_jshtml(default_mode='once'))